In [5]:
import json

### Apriamo un file json con tutti i risultati del campionato di seria A 2015/16

In [6]:
fp = open('datasets/serie_A/2015-16.json')
campionato = json.load(fp)

### Ispezioniamo il contenuto "trasformato" in strutture dati Python

In [ ]:
type(campionato)

In [ ]:
campionato.keys()

In [ ]:
campionato['name']

In [ ]:
type(campionato['rounds'])

In [ ]:
len(campionato['rounds'])

In [ ]:
prima_giornata = campionato['rounds'][0]

In [ ]:
type(prima_giornata)

In [ ]:
campionato['rounds'][0].keys()

In [ ]:
campionato['rounds'][0]['name']

In [ ]:
type(campionato['rounds'][0]['matches'])

In [ ]:
partita1 = campionato['rounds'][0]['matches'][0]

In [ ]:
type(partita1)

In [ ]:
partita1.keys()

In [ ]:
partita1

### Abbiamo la struttura
1. Un campionato è un dizionario con il nome (una stringa) e la lista delle giornate
2. Una giornata (un elemento della lista) è a sua volta un dizionario con il nome (1^ giornata, 2^ giornata, ....) e la lista degli incontri
3. Un incontro è un dizionario con 5 voci: 
   1. data (in cui si dell'incontro), 
   2. team1 (squadra di casa)
   3. team2 (squadra ospite)
   4. score1 (goal segnati dalla squadra di casa)
   5. score2 (goal segnati dalla squadra ospite)
4. Team1 e team2 sono a loro volta dei dizionari

### Proviamo innanzitutto a stampare (in modo "pulito") il programma della giornata

In [ ]:
for partita in campionato['rounds'][0]['matches']:
    print(partita['team1']['name'],'-',partita['team2']['name'])

### Primo obiettivo: ottenere una lista di coppie (codiceprogressivo,nomesquadra), con tutte le squadre partecipanti

#### Soluzioni non-pythoniche

In [ ]:
i = 0 # il codice parte da i+1

squadre_partecipanti = []
for partita in campionato['rounds'][0]['matches']:
    i += 1
    squadre_partecipanti.append((i, partita['team1']['name']))
    i += 1
    squadre_partecipanti.append((i, partita['team2']['name']))

In [ ]:
squadre_partecipanti

#### Un po' meglio ma sempre non pythonica

In [ ]:
def sq(partita, i):
    return [(i, partita['team1']['name']),(i+1, partita['team2']['name'])]

In [ ]:
sq(campionato['rounds'][0]['matches'][0],1)

In [ ]:
squadre_partecipanti = []
i = 1
for partita in campionato['rounds'][0]['matches']:
    squadre_partecipanti += sq(partita, i)
    i += 2

In [ ]:
squadre_partecipanti

### List comprehension

#### Un semplice esempio iniziale

In [ ]:
[i**2 for i in range(10)]

In [7]:
partite = [(p['team1']['name'],p['team2']['name']) for p in campionato['rounds'][0]['matches']]

In [ ]:
len(partite)

In [ ]:
partite

#### Dobbiamo innanzitutto passare da una lista di coppie (partite) ad una lista (o tupla) con le singole squadre

#### Possiamo usare la funzione reduce (pare non amata da Van Rossum)

#### reduce ha due parametri: una funzione di "aggregazione" dei dati (es: l'addizione intera da due numeri ne restituisce uno, la concatenazione di stringhe da due ne forma una, idem la "somma" di due liste....), e una lista di ogetti da "aggregare"

In [2]:
from functools import reduce

In [ ]:
def sum(x,y):
    return x+y # + si può applicare, con significati diversi, a diversi tipi di dato ==> operatore overloaded

In [ ]:
reduce(sum,[1,2,3,4,5])

In [ ]:
reduce(sum,['a','b','c','d','e'])

In [ ]:
reduce(sum,[(1,2),(3,4),(5,6),(7,8),(9,10)])

In [ ]:
reduce(sum,([1,2],[3,4],[5,6],[7,8],[9,10]))

#### Fare attenzione al tipo di dato restituito negli ultimi due casi

#### Per fare l'aggregazione, anziché usare una funzione opportunamente definita "prima" (nel nostro caso "sum"), possiamo utilizzare una funzione "anonima" mediante la "lambda notation". Si noti, infatti, che la funzione sum non ha altra utilità al di fuori dell'impiego che ne facciamo nel contesto di reduce. Dunque non serve darle un nome 

In [ ]:
reduce(lambda x,y: x+y,[1,2,3,4,5])

In [ ]:
reduce(lambda x,y: x+y, ['a','b','c','d','e'])

#### Possiamo ora ottenere l'elenco delle squadre

In [ ]:
elenco_squadre = reduce(lambda x,y: x+y,partite)

In [ ]:
elenco_squadre

#### ... e poi il risultato desiderato, usando enumerate (un iteratore che fornisce anche un contatore automatico)

In [ ]:
for index,value in enumerate(['a','b','c']):
    print(index,value)

In [ ]:
for index,value in enumerate(['a','b','c'], 15):
    print(index,value)

In [8]:
squadre = list(enumerate(reduce(lambda x,y: x+y, partite), 1))

#### Anche con un solo "comando":

In [ ]:
squadre = list(enumerate(reduce(lambda x,y: x+y,[(p['team1']['name'],p['team2']['name']) \
                                       for p in campionato['rounds'][0]['matches']]), 1))

In [17]:
squadre

[(1, 'Verona'),
 (2, 'Roma'),
 (3, 'Lazio'),
 (4, 'Bologna'),
 (5, 'Juventus'),
 (6, 'Udinese'),
 (7, 'Empoli'),
 (8, 'Chievo'),
 (9, 'Fiorentina'),
 (10, 'Milan'),
 (11, 'Frosinone'),
 (12, 'Torino'),
 (13, 'Inter'),
 (14, 'Atalanta'),
 (15, 'Palermo'),
 (16, 'Genoa'),
 (17, 'Sampdoria'),
 (18, 'Carpi'),
 (19, 'Sassuolo'),
 (20, 'Napoli')]

#### La lista è OK per un inserimento in un DB. Per altri scopi ci serve maggiormente un dizionario

In [ ]:
dizSquadre = {}
# partite = [(p['team1']['name'],p['team2']['name']) for p in campionato['rounds'][0]['matches']]
for id, squadra in enumerate(reduce(lambda x,y: x+y, partite),1):
   dizSquadre[squadra] = id

In [ ]:
dizSquadre

### Vogliamo ora generare una lista di tuple che descriva tutto il calendario
### Anche in questo caso le tuple devono contenere un id progressivo
### Devono poi contenere il numero di giornata, l'indicazione di andata o ritorno (A/R) e naturalmente gli identificatori delle squadre 

In [ ]:
def locali(p):
    return p['team1']['name']
def ospiti(p):
    return p['team2']['name']
def retiLocali(p):
    return p['score1']
def retiOspiti(p):
    return p['score2']

In [ ]:
calendario = []
risultati = []
idCal = 1
idRis = 1
for numGiornata, giornata in enumerate(campionato['rounds']):
    for numPartita, partita in enumerate(giornata['matches']):
        sqloc = dizSquadre[locali(partita)]
        sqosp = dizSquadre[ospiti(partita)]
        offset = numGiornata*10+numPartita
        calendario.append((idCal+offset,\
                           numGiornata%19+1,\
                           ['A','R'][numGiornata>18],\
                           sqloc,sqosp))
        risultati.append((idRis+offset,\
                         idCal+offset,
                         retiLocali(partita),
                         retiOspiti(partita)))
                         
        #print(idIniziale+(i-idIniziale)*10+j,(i-idIniziale)%19+1,['A','R'][(i-idIniziale)>18],h,v)

In [ ]:
calendario[:10]

In [ ]:
risultati[:10]

In [ ]:
campionato['rounds'][0]

## Database connection
### pip3 install PyMySQL

In [8]:
import pymysql

### Supponiamo di avere un utente mysql "testuser" sul server locale, con password di accesso "testpasswd" e tutti i diritti sul db "testdb". Il db è inizialmente vuoto

In [9]:
db = pymysql.connect("localhost","testuser","testpasswd","testdb" )

In [10]:
db

### Prepariamo un cursore per l'accesso al db. Un cursore gioca un ruolo analogo ad un iteratore Python: anziché avere tutti i dati in memoria, il cursore consente l'accesso "pezzo per pezzo". Si pensi alla differenza fra range() in python 2 e python 3.

In [11]:
cursor = db.cursor()

In [12]:
cursor

### Creiamo la tabella Squadre, assicurandoci prima che non sia già presente

In [13]:
query = "DROP TABLE IF EXISTS Squadre;"

In [14]:
cursor.execute(query)

/home/mauro/.local/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1051, "Unknown table 'testdb.Squadre'")
  result = self._query(query)


0

In [15]:
query = "CREATE TABLE Squadre (id INT(11) NOT NULL AUTO_INCREMENT, " +\
                              "nome VARCHAR(16) NOT NULL, " +\
                              "PRIMARY KEY ( id ));";

In [16]:
cursor.execute(query)

0

### Adesso inseriamo i dati nella tabella Squadre appena creata. Possiamo ripetere la stessa query con i dati provenienti da una lista (o da un enumeratore/iteratore)

In [18]:
query = "INSERT INTO Squadre VALUES (%s, %s);"

In [19]:
try:
    # Esecuzione della query
    cursor.executemany(query, squadre)
    # Commit delle modifiche sul database, se tutto OK
    db.commit()
except:
    print("Doing rollback ....")
    # Rollback in caso di errore
    db.rollback()

In [ ]:
try:
    cursor.execute("delete from Squadre;")
    db.commit()
except:
    db.rollback()

In [ ]:
try:
    cursor.executemany(query, enumerate(elenco_squadre,1))
    db.commit()
except:
    print("Doing rollback ....")
    db.rollback()

### Inseriamo ora le altre tabelle e i dati che abbiamo preparato

In [ ]:
query = "DROP TABLE IF EXISTS Calendario;"

In [ ]:
cursor.execute(query)

In [ ]:
query = "CREATE TABLE Calendario (id INT(11) NOT NULL AUTO_INCREMENT, " +\
                              "giornata int(2) NOT NULL, " +\
                              "AR CHAR(1) NOT NULL, " +\
                              "data VARCHAR(10) NOT NULL,"
                              "locali int(11) NOT NULL, " +\
                              "ospiti int(11) NOT NULL, " +\
                              "PRIMARY KEY ( id ));";

In [ ]:
cursor.execute(query)

In [ ]:
query = "INSERT INTO Calendario VALUES (%s, %s, %s, %s, %s);"

In [ ]:
try:
    cursor.executemany(query, calendario)
    db.commit()
except:
    print("Doing rollback ....")
    db.rollback()

In [ ]:
query = "DROP TABLE IF EXISTS Risultati;"

In [ ]:
cursor.execute(query)

In [ ]:
query = "CREATE TABLE Risultati (id INT(11) NOT NULL AUTO_INCREMENT, " +\
                               "partita int(11) NOT NULL, " +\
                               "retiLocali int(2) NOT NULL, " +\
                               "retiOspiti int(2) NOT NULL, " +\
                               "PRIMARY KEY ( id ));";

In [ ]:
cursor.execute(query)

In [ ]:
query = "INSERT INTO Risultati VALUES (%s, %s, %s, %s);"

In [ ]:
try:
    cursor.executemany(query, risultati)
    db.commit()
except:
    print("Doing rollback ....")
    db.rollback()

### Risultato di Samp - Genoa

In [ ]:
query = "SELECT Sq1.nome AS Locali, Sq2.nome AS Ospiti, retiLocali, retiOspiti " + \
        "FROM Calendario INNER JOIN Squadre AS Sq1 ON Calendario.locali = Sq1.id " + \
        "INNER JOIN Squadre AS Sq2 ON Calendario.ospiti = Sq2.id " + \
        "INNER JOIN Risultati ON Calendario.id = Risultati.partita " + \
        "INNER JOIN Campionati ON Calendario.campionato = Campionati.id " + \
        "WHERE Sq1.nome = 'Sampdoria' AND Sq2.nome = 'Genoa';"

### Risultati della terza gionata di ritorno del campionato di serie A 2015/16

In [ ]:
query = "SELECT Sq1.nome AS Locali, Sq2.nome AS Ospiti, retiLocali, retiOspiti " + \
        "FROM Calendario INNER JOIN Squadre AS Sq1 ON Calendario.locali = Sq1.id " + \
        "INNER JOIN Squadre AS Sq2 ON Calendario.ospiti = Sq2.id " + \
        "INNER JOIN Risultati ON Calendario.id = Risultati.partita " + \
        "INNER JOIN Campionati ON Calendario.campionato = Campionati.id " + \
        "WHERE Campionati.nome = 'Serie A 2015/16' AND Calendario.giornata=3 AND Calendario.AR = 'R';"

### Il cammino della Juventus verso il titolo

In [ ]:
query = "SELECT Giornata, AR, Sq1.nome AS Locali, Sq2.nome AS Ospiti, retiLocali, retiOspiti " + \
        "FROM Calendario INNER JOIN Squadre AS Sq1 ON Calendario.locali = Sq1.id " + \
        "INNER JOIN Squadre AS Sq2 ON Calendario.ospiti = Sq2.id " + \
        "INNER JOIN Risultati ON Calendario.id = Risultati.partita " + \
        "INNER JOIN Campionati ON Calendario.campionato = Campionati.id " + \
        "WHERE Campionati.nome = 'Serie A 2015/16' AND (Sq1.nome = 'Juventus' OR Sq2.nome = 'Juventus');"

### La generazione della classifica è (relativamente) challenging

### Definiamo preliminarmente due viste, una con tutti i risultati e una che "conta" i punti di ciascuna squadra in tutte le partite

In [ ]:
query = "CREATE VIEW Risultati_Serie_A_201516 AS " + \
        "SELECT Cal.giornata AS giornata, Cal.Ar AS girone, SC.nome AS locali, SO.nome AS ospiti, " + \
        "       Ris.retiLocali AS retiLocali, Ris.retiOspiti AS retiOspiti " + \
        "FROM Campionati AS Cam INNER JOIN Calendario AS Cal ON Cal.campionato = Cam.id " + \
        "                       INNER JOIN Squadre AS SC ON Cal.locali = SC.id " + \
        "                       INNER JOIN Squadre AS SO ON Cal.ospiti = SO.id " + \
        "                       LEFT JOIN Risultati AS Ris ON Ris.partita = Cal.id " + \
        "WHERE Cam.id=20 ORDER BY girone ASC, Giornata ASC;"

In [ ]:
query = "CREATE VIEW Risultati_Serie_A_201516_2 AS " + \
        "SELECT Risultati_Serie_A_201516.giornata AS giornata, " + \
        "       Risultati_Serie_A_201516.locali AS squadra, 'C' AS casa_fuori, " + \
        "       Risultati_Serie_A_201516.retiLocali AS reti_fatte, " + \
        "       Risultati_Serie_A_201516.retiOspiti AS reti_subite, " + \
        " (CASE WHEN (Risultati_Serie_A_201516.retiLocali > Risultati_Serie_A_201516.retiOspiti) THEN 3 " + \
        "       WHEN (Risultati_Serie_A_201516.retiLocali = Risultati_Serie_A_201516.retiOspiti) THEN 1 " + \
        "       ELSE 0 END) AS Punti " + \
        "FROM Risultati_Serie_A_201516 " + \
        "WHERE (Risultati_Serie_A_201516.retiLocali IS NOT NULL) " + \
        "UNION " + \
        "SELECT Risultati_Serie_A_201516.giornata AS giornata, " + \
        "       Risultati_Serie_A_201516.ospiti AS squadra, 'F' AS casa_fuori, " + \
        "       Risultati_Serie_A_201516.retiOspiti AS reti_fatte, " + \
        "       Risultati_Serie_A_201516.retiLocali AS reti_subite, " + \
        " (CASE WHEN (Risultati_Serie_A_201516.retiOspiti > Risultati_Serie_A_201516.retiLocali) THEN 3 " + \
        "       WHEN (Risultati_Serie_A_201516.retiLocali = Risultati_Serie_A_201516.retiOspiti) THEN 1 " + \
        "       ELSE 0 END) AS Punti " + \
        "FROM Risultati_Serie_A_201516 " + \
        "WHERE (Risultati_Serie_A_201516.retiLocali IS NOT NULL);"

### Possiamo ora generare la classifica

In [ ]:
query = "SELECT squadra, " + \
        "       SUM(punti) AS Pt, COUNT(*) as G, " + \
        "       SUM(CASE WHEN punti=3 THEN 1 ELSE 0 END) AS V, " + \
        "       SUM(CASE WHEN punti=1 THEN 1 ELSE 0 END) AS N, " + \
        "       SUM(CASE WHEN punti=0 THEN 1 ELSE 0 END) AS P, " + \
        "       SUM(reti_fatte) AS GF, " + \
        "       SUM(reti_subite) AS GS, " + \
        "       SUM(reti_fatte)-SUM(reti_subite) AS Diff " + \
        "FROM Risultati_Serie_A_201516_2 " + \
        "GROUP BY squadra " + \
        "ORDER BY 2 DESC, 9 DESC, 1 ASC;"